### Hillman(key exchange)

In [307]:
# functions: generate_key(p,q), generate_shared_key(pub,priv,p)
# p,q,priv,pub
import random
def generate_key(p,q):
    priv = random.randint(2,p-2) # should be less than p
    pub = pow(q,priv,p)  #(q^priv)%p
    return pub,priv

def generate_shared_key(priv,pub,p):
    return pow(pub,priv,p)
p = 17 #prime number
q = 5 #primitive root of p

#generate pub and priv keys
#userA
A_pub,A_priv = generate_key(p,q)
#userB
B_pub, B_priv = generate_key(p,q)

#generate shared keys for both the users
#userA
A_key = generate_shared_key(A_priv,B_pub,p)
#userB
B_key = generate_shared_key(B_priv,A_pub,p)

print("USER A : PUBLIC KEY = {} AND PRIVATE KEY = {} AND SHARED KEY = {}".format(A_pub,A_priv,A_key))
print("USER B : PUBLIC KEY = {} AND PRIVATE KEY = {} AND SHARED KEY = {}".format(B_pub,B_priv,B_key))

USER A : PUBLIC KEY = 2 AND PRIVATE KEY = 6 AND SHARED KEY = 9
USER B : PUBLIC KEY = 7 AND PRIVATE KEY = 15 AND SHARED KEY = 9


### RSA

In [316]:
# generate_keys(p,q), encrypt(msg,public_key), decrypt(enc_msg,private_key), gcd(a,b)
import random
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

def generate_keys(p, q):
    n = p * q
    phi = (p - 1) * (q - 1)
    
    e = random.randrange(2, phi)
    while gcd(e, phi) != 1:
        e = random.randrange(2, phi)
    
    d = pow(e, -1, phi)
    return ((e, n), (d, n))

def encrypt(msg, public):
    enc = []
    e, n = public
    for c in msg:
        enc.append(pow(ord(c), e, n))
    return enc

def decrypt(enc, private):
    dec = []
    d, n = private
    for c in enc:
        dec.append(chr(pow(c, d, n)))
    return ''.join(dec)

p, q = 43, 53 
public_key, private_key = generate_keys(p, q)
message = input()
encrypted_message = encrypt(message, public_key)
decrypted_message = decrypt(encrypted_message, private_key)

print("enc: ",encrypted_message,"     dec:",decrypted_message)


hello world
enc:  [2098, 103, 75, 75, 1858, 2244, 227, 1858, 843, 75, 1176]      dec: hello world


### Transposition

In [336]:
#enc -> row col, col row
#dec -> col row, row col

def encrypt(text,key):
    # add padding
    while len(text)% key !=0:
        text+=' '
    
    cols = len(text) // key
    rows = key
    grid=[]
    for _ in range(rows):
        grid.append(['']*cols)
    
    i = 0
    for row in range(rows):
        for col in range(cols):
            grid[row][col] = text[i]
            i+=1
    res = ""
    for col in range(cols):
        for row in range(rows):
            res+=grid[row][col]
    return res


def decrypt(text,key):
    cols = len(text) // key
    rows = key
    grid=[]
    for _ in range(rows):
        grid.append(['']*cols)
    
    i = 0
    for col in range(cols):
        for row in range(rows):
            grid[row][col] = text[i]
            i+=1
    res = ""
    for row in range(rows):
        for col in range(cols):
            res+=grid[row][col]
    return res

text = "hello"
key = 3
enc = encrypt(text,key)
dec = decrypt(enc,key)

print("enc:",enc)
print()
print("--------------------------------------------------------------------------------------------------------------")
print("dec:",dec)

enc: hloel 

--------------------------------------------------------------------------------------------------------------
dec: hello 


### DES

In [310]:
# apply_table
def apply_table(inp,table):
    res = ""
    for i in table:
        res+=inp[i-1]
    return res
    
# lcs function
def lcs(data):
    return data[1:] + data[0]

def xor(a, b):
    res = ""
    for i in range(len(a)):
        if a[i] == b[i]:
            res += "0"
        else:
            res += "1"
    return res


def apply_sbox(s, data): 
    row = int("0b" + data[0] + data[-1], 2) # taking the first and last element as row and into decimal
    col = int("0b" + data[1:3], 2) 
    
    
    return bin(s[row][col])[2:] 

#function
def function(EP,s0,s1,key,msg):
    left = msg[:4]
    right = msg[4:]
    temp = apply_table(right,EP)
    temp = xor(temp,key)
    l = apply_sbox(s0,temp[:4])
    r = apply_sbox(s1,temp[4:])
    l = "0"*(2-len(l)) + l
    r = "0"*(2-len(r)) + r
    temp = apply_table(l+r,p4_table)
    temp = xor(left,temp)
    return temp + right
    
#p10,p8,p4,IP,EP,s0,s1
p8_table = [6, 3, 7, 4, 8, 5, 10, 9]
p10_table = [3, 5, 2, 7, 4, 10, 1, 9, 8, 6]
p4_table = [2, 4, 3, 1]
IP = [2, 6, 3, 1, 4, 8, 5, 7]
IP_inv = [4, 1, 3, 5, 7, 2, 8, 6]
EP = [4, 1, 2, 3, 2, 3, 4, 1]
s0 = [[1, 0, 3, 2], [3, 2, 1, 0], [0, 2, 1, 3], [3, 1, 3, 2]]
s1 = [[0, 1, 2, 3], [2, 0, 1, 3], [3, 0, 1, 0], [2, 1, 0, 3]]

# key generation (key1 and key2)
key = input("10 bit key: ")
temp = apply_table(key,p10_table)
left = temp[:5]
right = temp[5:]
left = lcs(left)
right = lcs(right)
key1 = apply_table(left+right,p8_table)
left = lcs(left)
right = lcs(right)
left = lcs(left)
right = lcs(right)
key2 = apply_table(left + right, p8_table)

# encryption
msg = input("message 8 bit: ")
temp = apply_table(msg,IP)
temp = function(EP,s0,s1,key1,temp)
temp = temp[4:] + temp[:4] #swap
temp = function(EP,s0,s1,key2,temp)
CT = apply_table(temp,IP_inv)
print("CT:",CT)

temp = apply_table(CT,IP)
temp = function(EP,s0,s1,key2,temp)
temp = temp[4:] + temp[:4] #swap
temp = function(EP,s0,s1,key1,temp)
PT = apply_table(temp,IP_inv)
print("PT:",PT)

10 bit key: 1011010101
message 8 bit: 10101010
CT: 10110010
PT: 10101010


### Scheduling

In [331]:
arr = [['a', 2, 20], 
       ['b', 2, 15],
       ['c', 1, 10],
       ['d', 4, 5],
       ['e', 3, 1]]
t = 4
n = len(arr)
slots = [False]*t
result = ['-1']*t

for i in range(n):
    for j in range(n-1-i):
        if arr[j][2] < arr[j+1][2]:
            arr[j],arr[j+1] = arr[j+1],arr[j]

for i in range(len(arr)):
    for j in range(min(arr[i][1]-1,t-1),-1,-1):
        if slots[j] == False:
            slots[j] = True
            result[j] = arr[i][0]
            break
            
print(result)

['b', 'a', 'e', 'd']


### Nqueens

In [334]:
def is_safe(board, row, col):
    n = len(board)
    
    # Check the row
    for i in range(col):
        if board[row][i] == 1:
            return False
        
    # Check the upper diagonal
    r, c = row, col
    while r >= 0 and c >= 0:
        if board[r][c] == 1:
            return False
        r -= 1
        c -= 1

    # Check the lower diagonal
    r, c = row, col
    while r < n and c >= 0:
        if board[r][c] == 1:
            return False
        r += 1
        c -= 1

    # If no conflicts found, it's safe to place a queen
    return True

def backtrack(board, col, solutions):
    n = len(board)
    
    # If all queens are placed, a valid solution is found
    if col == n:
        solutions.append([row[:] for row in board])
        return

    # Explore all possible positions in the current column
    for row in range(n):
        if is_safe(board, row, col):
            board[row][col] = 1  # Place a queen

            # Recursively move to the next column
            backtrack(board, col + 1, solutions)

            board[row][col] = 0  # Remove the queen (backtrack)

def solve_nqueens(n):
    board = [[0] * n for _ in range(n)]
    solutions = []
    backtrack(board, 0, solutions)
    return solutions

# Example usage
n = 4
solutions = solve_nqueens(n)

print(f"Total solutions for {n}-queens problem: {len(solutions)}")
for i, solution in enumerate(solutions):
    print(f"Solution {i+1}:")
    for row in solution:
        print(row)
    print()


Total solutions for 4-queens problem: 2
Solution 1:
[0, 0, 1, 0]
[1, 0, 0, 0]
[0, 0, 0, 1]
[0, 1, 0, 0]

Solution 2:
[0, 1, 0, 0]
[0, 0, 0, 1]
[1, 0, 0, 0]
[0, 0, 1, 0]



### ChatBot

In [313]:
import datetime
greetings = ['Hello!','Hi!']
salutations = ['Bye!','See you soon!','Have a good day!']
others = {"weather" : ["sunny","chilly","rainy"],
          "name" : ["My name is HarshBot","Myself HarshBot","People call me HarshBot"]
         }
def greet():
    print(random.choice(greetings))
def farewell():
    print(random.choice(salutations))
def date():
    print("The date is",str(datetime.datetime.now())[:10])
def time():
     print("The time is",str(datetime.datetime.now())[11:16])
def process(inp):
    if "hello" in inp.lower():
        greet()
    elif "bye" in inp.lower():
        farewell()
        return True
    elif "date" in inp.lower():
        date()
    elif "time" in inp.lower():
        time()
    else:
        found_response = False
        for key,value in others.items():
            if key in inp.lower():
                print(random.choice(value))
                found_response = True
                break
        if not found_response:
            print("idk")
            
    return False      

finished = False
while not finished:
    inp = input(">    ")
    finished = process(inp)

>    hello
Hello!
>    weather
sunny
>    time
The time is 00:23
>    date
The date is 2023-05-27
>    name
Myself HarshBot
>    lala
idk
>    bye
Have a good day!


### Expert System

In [314]:
rules = {
    "rule1": {
        "condition" : lambda data : data["attendance"] >=7.5 and data["creative"] >= 7,
        "output" : "Excellent Employee!"
    },
    "rule2" : {
        "condition" : lambda  data : data["attendance"] >= 5 and data["creative"] >=6 ,
        "output" : "Average Employee"
    },
    "rule3" : {
        "condition" : lambda data: data["attendance"] <5 and data["creative"] <5,
        "output" : "Poor"
    }
}
def evaluate(data):
    for rule in rules.values():
        if rule['condition'](data):
            return rule['output']
    return "Need more details"

data = {
    "attendance" : int(input("attendance: ")),
    "creative" : int(input("creative: "))
    }
     
perf = evaluate(data)
print(perf)    

     

attendance: 9
creative: 10
Excellent Employee!


### BFS AND DFS IN JAVA

### Astar Algo

In [328]:
def astar(start,goal,grid):
    OL,CL=[],[]
    OL.append((start,None,0,abs(start[0]-goal[0])+abs(start[1]-goal[1])))
    #processing
    while OL:
        #lowest f 
        curr_node = min(OL,key = lambda node:node[2])
        #add in CL
        CL.append(curr_node)
        OL.remove(curr_node)
        #is goal?
        if curr_node[0] == goal:
            path = []
            while curr_node:
                path.append(curr_node[0])
                curr_node = curr_node[1]
            return path[::-1]
        #generate neighbors
        neighbors=[]
        x,y = curr_node[0]
        moves = [(0,1),(1,0),(-1,0),(0,-1)]
        for move in moves:
            new_x = x + move[0]
            new_y = y + move[1]
            if (0<= new_x < len(grid) and 0 <= new_y < len(grid) and grid[new_x][new_y]!="#"):
                neighbors.append(((new_x,new_y),curr_node,curr_node[2]+1,abs(new_x - goal[0])+abs(new_y - goal[1])))
        #process each neighbor
        for neighbor in neighbors:
            if neighbor in CL:
                continue
            found_duplicate = False
            for node in OL:
                if (neighbor[0]==node[0] and neighbor[2] == node[2]):
                    found_duplicate = True
                    break
            if not found_duplicate:
                OL.append(neighbor)
    return None

grid = [
    ['S','.','#','.','.'],
    ['.','.','.','.','.'],
    ['.','#','#','.','.'],
    ['.','.','G','.','.'],
    ['.','.','.','.','.']
    ]

start = (0, 0)  # Starting position
goal = (3, 2)  # Goal position

path = astar(start, goal, grid)
if path:
    print("Path found:")
    for position in path:
        print(position)
else:
    print("No path found.")


Path found:
(0, 0)
(1, 0)
(2, 0)
(3, 0)
(3, 1)
(3, 2)
